In [ ]:
from jetbot import Camera, bgr8_to_jpeg
import traitlets
import ipywidgets as widgets
from IPython.display import display
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import time
from jetbot import Robot
#-------------
# functions
#------------
def sign(input):
    return input / abs(input) if input != 0 else 0  # 避免除以 0

def maxNum(input):
    if abs(input) > 100:
        return 100 * sign(input)
    else:
        return input

def SpeedControl(s_in, d_in):
    L = R = 0
    S = maxNum(s_in) 
    D = maxNum(d_in)/2    

    L = S + D
    R = S - D

    L = maxNum(L)
    R = maxNum(R)
    L = L / 100
    R = R / 100
    #------------
    # 簡化你的速度限制
    L = max(min(L, 1.0), -1.0)
    R = max(min(R, 1.0), -1.0)
        
    robot.set_motors(L, R)
    return (L, R)

print('Cell 1: 函式庫與輔助函數已載入')

Cell 1: 函式庫與輔助函數已載入


In [61]:
# --- 初始化 JetBot 元件 ---
# 你的 .ipynb 檔案顯示有 RuntimeError，這通常是核心問題
# 試著在啟動此儲存格前，點擊 Jupyter 上方的 "Kernel" -> "Restart Kernel"
try:
    camera.stop() # 先停止可能在運行的相機
except NameError:
    pass # 第一次執行時 camera 還沒定義

robot = Robot()
camera = Camera.instance()

print('Cell 2: Robot 與 Camera 已初始化')

# --- 創建 IPyWidgets 小工具 ---
image_widget = widgets.Image(format='jpeg', width=400, height=300)
binary_widget = widgets.Image(format='jpeg', width=400, height=300)

# ROI (Region of Interest)
roi_x, roi_y, roi_w, roi_h = 100, 100, 10, 10 
mean_label = widgets.Label()

# --- 顏色滑桿 (黃色) ---
r_lower = widgets.IntSlider(min=0, max=255, value=0, description='R Lower')
r_upper = widgets.IntSlider(min=0, max=255, value=175, description='R Upper')
g_lower = widgets.IntSlider(min=0, max=255, value=190, description='G Lower')
g_upper = widgets.IntSlider(min=0, max=255, value=255, description='G Upper')
b_lower = widgets.IntSlider(min=0, max=255, value=140, description='B Lower')
b_upper = widgets.IntSlider(min=0, max=255, value=255, description='B Upper')

# --- 控制滑桿 ---
min_area_slider = widgets.IntSlider(value=500, min=0, max=10000, step=50, description='最小面積')
txt_speed = widgets.BoundedIntText(value=15, min=0, max=100, step=1, description='Speed')
tracking_toggle = widgets.ToggleButton(value=False, description='啟用尋跡', button_style='success')

# 顯示所有小工具
display(widgets.HBox([image_widget, binary_widget]), mean_label)
display(widgets.VBox([
    widgets.HBox([r_lower, r_upper]),
    widgets.HBox([g_lower, g_upper]),
    widgets.HBox([b_lower, b_upper]),
    min_area_slider,
    txt_speed,
    tracking_toggle
]))
print('Cell 2: Widgets 已創建')

Cell 2: Robot 與 Camera 已初始化


Label(value='')

Cell 2: Widgets 已創建


In [ ]:
# -----------------------------------------------
# 5. 全域變數與狀態機 (取代你的 cell 3 開頭)
# -----------------------------------------------
frame_count = 0
process_every_n_frames = 2  # 每 N 幀處理一次影像

# 避障狀態機 (取代 time.sleep)
avoidance_state = 0         # 0=不避障, 1=右轉, 2=前進, 3=左轉
avoidance_timer = 0.0
AVOID_TURN_TIME = 0.4       # 右轉/左轉 0.5 秒
AVOID_STRAIGHT_TIME = 0.8   # 直行 1 秒

# 避障 (紅色) 的顏色範圍 (R, G, B)
OBSTACLE_LOWER_RGB = np.array([71, 44, 141])
OBSTACLE_UPPER_RGB = np.array([250, 134, 255])

print('Cell 3: 全域變數已設定')

# -----------------------------------------------
# 6. 核心影像處理函數 (取代你的 update_images)
# -----------------------------------------------
def update_images(change):
    global frame_count, avoidance_state, avoidance_timer
    
    frame = change['new']
    if frame is None:
        return

    # --- 效能控制 ---
    frame_count += 1
    if frame_count % process_every_n_frames != 0:
        return

    # 讀取當前速度
    speed = txt_speed.value
    
    # 獲取影像中心
    height, width, _ = frame.shape
    frame_center_x = width // 2

    # --- 狀態機：處理避障動作 ---
    # (這部分取代了你的 time.sleep() 邏輯)
    current_time = time.time()
    
    if avoidance_state == 1: # 狀態1: 右轉
        SpeedControl(speed, 30) 
        if current_time - avoidance_timer > AVOID_TURN_TIME:
            avoidance_state = 2 # 進入狀態2
            avoidance_timer = current_time
            robot.stop()
        return # 避障中，不執行後續偵測
    
    elif avoidance_state == 2: # 狀態2: 直行
        SpeedControl(speed, 0)
        if current_time - avoidance_timer > AVOID_STRAIGHT_TIME:
            avoidance_state = 3 # 進入狀態3
            avoidance_timer = current_time
        return
    elif avoidance_state == 3: # 狀態3: 左轉
        SpeedControl(speed, -25) 
        if current_time - avoidance_timer > AVOID_TURN_TIME:
            avoidance_state = 4 # 進入狀態2
            avoidance_timer = current_time
            robot.stop()
        return # 避障中，不執行後續偵測
    elif avoidance_state == 4: # 狀態2: 直行
        SpeedControl(speed, 0)
        if current_time - avoidance_timer > AVOID_STRAIGHT_TIME:
            avoidance_state = 5 # 進入狀態3
            avoidance_timer = current_time
            
        return
    elif avoidance_state == 5: # 狀態3: 左轉
        SpeedControl(speed, -30)
        if current_time - avoidance_timer > AVOID_TURN_TIME:
            avoidance_state = 6 # 進入狀態2
            avoidance_timer = current_time
            robot.stop()
        return # 避障中，不執行後續偵測
    elif avoidance_state == 6: # 狀態2: 直行
        SpeedControl(speed, 0)
        if current_time - avoidance_timer > AVOID_STRAIGHT_TIME - 0.6:
            avoidance_state = 7 # 進入狀態3
            avoidance_timer = current_time
        return
    elif avoidance_state == 7: # 狀態3: 左轉
        SpeedControl(speed, 30)
        if current_time - avoidance_timer > AVOID_TURN_TIME + 0:
            avoidance_state = 0 # 避障結束
            robot.stop() # 避障完畢先停止
        return

    # --- 影像處理 (繪圖用) ---
    bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.rectangle(bgr_frame, (roi_x, roi_y), (roi_x + roi_w, roi_y + roi_h), (0, 0, 255), 2) # 繪製 ROI

    # (可選) 顯示 ROI 區域的平均值
    roi_frame = frame[roi_y:roi_y + roi_h, roi_x:roi_x + roi_w]
    mean_rgb = roi_frame.mean(axis=(0, 1))
    mean_label.value = f"ROI 平均 RGB 值：R={mean_rgb[0]:.2f}, G={mean_rgb[1]:.2f}, B={mean_rgb[2]:.2f}"

    # --- 邏輯 1: 偵測障礙物 (紅色) ---
    mask_obstacle = cv2.inRange(frame, OBSTACLE_LOWER_RGB, OBSTACLE_UPPER_RGB)
    contours_obs, _ = cv2.findContours(mask_obstacle, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours_obs:
        largest_cnt_obs = max(contours_obs, key=cv2.contourArea)
        M_obs = cv2.moments(largest_cnt_obs)
        if M_obs['m00'] > min_area_slider.value: # 檢查是否大於最小面積
            cx_obs = int(M_obs['m10'] / M_obs['m00'])
            cy_obs = int(M_obs['m01'] / M_obs['m00'])
            
            # 在影像上用藍色畫出障礙物
            cv2.circle(bgr_frame, (cx_obs, cy_obs), 10, (0, 0, 255), 2) 
            
            # 觸發避障 (如果啟用追蹤且障礙物夠近)
            if cy_obs > 70 and tracking_toggle.value:
                avoidance_state = 1      # 啟動狀態機
                robot.set_motors(-0.2, -0.2)
                time.sleep(0.4)
                robot.stop()
                avoidance_timer = time.time()
                # 更新影像並立刻返回
                _, frame_encoded = cv2.imencode('.jpg', cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB))
                image_widget.value = frame_encoded.tobytes()
                return 

    # --- 邏輯 2: 偵測追蹤目標 (黃色) ---
    # (如果沒有觸發避障，才會執行到這裡)
    
    # *** 修正BUG：使用 [R, G, B] 順序 ***
    lower_target = np.array([r_lower.value, g_lower.value, b_lower.value])
    upper_target = np.array([r_upper.value, g_upper.value, b_upper.value])
    mask_target = cv2.inRange(frame, lower_target, upper_target)
    
    contours_target, _ = cv2.findContours(mask_target, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    closest_centroid = None
    if contours_target:
        largest_cnt_target = max(contours_target, key=cv2.contourArea)
        M_target = cv2.moments(largest_cnt_target)
        if M_target['m00'] > min_area_slider.value:
            cx = int(M_target['m10'] / M_target['m00'])
            cy = int(M_target['m01'] / M_target['m00'])
            closest_centroid = (cx, cy)
            
            # 在影像上用綠色畫出追蹤目標
            cv2.circle(bgr_frame, closest_centroid, 10, (0, 255, 0), -1)

    # --- 馬達控制 (追蹤) ---
    if tracking_toggle.value:
        if closest_centroid:
            # 計算誤差
            error = closest_centroid[0] - frame_center_x
            d_in = (error / width) * 100  # 標準化誤差
            SpeedControl(speed, d_in)
        else:
            # 沒看到目標，停止
            robot.stop()
    else:
        # 沒啟用追蹤，停止
        robot.stop()
        
    # --- 更新影像小工具 ---
    _, frame_encoded = cv2.imencode('.jpg', cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB))
    image_widget.value = frame_encoded.tobytes()
    
    # 顯示追蹤目標的二值化影像
    mask_rgb = cv2.cvtColor(mask_target, cv2.COLOR_GRAY2RGB)
    _, binary_encoded = cv2.imencode('.jpg', mask_rgb)
    binary_widget.value = binary_encoded.tobytes()

print('Cell 3: 影像處理函數已定義')

Cell 3: 全域變數已設定
Cell 3: 影像處理函數已定義


In [63]:
camera.observe(update_images, names='value')
print("Cell 4: 攝影機已啟動，開始處理影像...")

Cell 4: 攝影機已啟動，開始處理影像...


In [45]:
# 執行此儲存格來停止一切
try:
    camera.unobserve(update_images, names='value')
    camera.stop()
    robot.stop()
    print('Cell 5: 攝影機與機器人均已停止')
except Exception as e:
    print(f"停止時發生錯誤 (可能未啟動): {e}")

Cell 5: 攝影機與機器人均已停止


In [ ]:
import time
import cv2
import numpy as np
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg
import ipywidgets as widgets
from IPython.display import display

# ==========================================
# 1. 參數設定區 (在此調整速度與靈敏度)
# ==========================================

# --- 影像處理參數 ---
process_every_n_frames = 3    # 降頻處理：每 3 幀處理一次，減輕運算負擔 [cite: 562]
min_area_threshold = 500      # 最小色塊面積，太小的雜訊忽略

# --- 避障顏色 (紅色障礙物) ---
# 依據現場光線調整這裡，或是使用投影片提到的黃色小鴨 [cite: 416]
OBSTACLE_LOWER_RGB = np.array([71, 44, 141]) 
OBSTACLE_UPPER_RGB = np.array([250, 134, 255])

# --- 避障動作參數 (時間與速度) ---
AVOID_TURN_TIME = 0.45        # 轉彎耗時 (秒)
AVOID_STRAIGHT_TIME = 0.8     # 直行耗時 (秒)
OBS_TRIGGER_Y = 70            # 障礙物多近才觸發 (Y軸像素，越大代表越近)

SPEED_NORMAL = 15             # 平常巡線速度
SPEED_AVOID_TURN = 30         # 避障轉彎力度 (差速)
SPEED_AVOID_STRAIGHT = 15     # 避障直行速度

# ==========================================
# 2. 全域變數與狀態機初始
# ==========================================
frame_count = 0
avoidance_state = 0           # 0=正常巡線, 1=右轉閃避, 2=直行, 3=左轉回正...
avoidance_timer = 0.0         # 計時器

# 初始化 Robot 與 Camera
try:
    camera.stop() # 確保先關閉舊的
except:
    pass
robot = Robot()
camera = Camera.instance()

# ==========================================
# 3. 輔助函數 (你的原始邏輯)
# ==========================================
def sign(input):
    return input / abs(input) if input != 0 else 0

def maxNum(input):
    if abs(input) > 100:
        return 100 * sign(input)
    else:
        return input

def SpeedControl(s_in, d_in):
    # 這是你的核心馬達控制函數 [cite: 467]
    S = maxNum(s_in) 
    D = maxNum(d_in) / 2    
    L = S + D
    R = S - D
    # 轉換為 -1.0 ~ 1.0 格式
    L = max(min(L / 100.0, 1.0), -1.0)
    R = max(min(R / 100.0, 1.0), -1.0)
    robot.set_motors(L, R)

# ==========================================
# 4. 核心影像處理 (主迴圈)
# ==========================================
def update_images(change):
    global frame_count, avoidance_state, avoidance_timer
    
    # --- A. 取得影像與效能管理 ---
    frame = change['new']
    if frame is None: return
    
    frame_count += 1
    if frame_count % process_every_n_frames != 0:
        return # 跳過這一幀不處理

    # 取得影像中心點與控制項數值
    height, width, _ = frame.shape
    frame_center_x = width // 2
    current_speed = txt_speed.value
    current_time = time.time()
    
    # 準備繪圖用的影像 (RGB轉BGR供OpenCV繪圖)
    bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # ==========================================
    # --- B. 狀態機：避障模式 (最高優先級) ---
    # ==========================================
    # 邏輯：如果不為 0，代表正在執行避障動作，直接覆管馬達，不進行巡線
    
    if avoidance_state != 0:
        time_elapsed = current_time - avoidance_timer
        
        if avoidance_state == 1:   # 步驟1: 右轉 (閃避)
            SpeedControl(SPEED_AVOID_STRAIGHT, SPEED_AVOID_TURN) 
            if time_elapsed > AVOID_TURN_TIME:
                avoidance_state = 2
                avoidance_timer = current_time
                
        elif avoidance_state == 2: # 步驟2: 直行 (繞過側面)
            SpeedControl(SPEED_AVOID_STRAIGHT, 0)
            if time_elapsed > AVOID_STRAIGHT_TIME:
                avoidance_state = 3
                avoidance_timer = current_time
                
        elif avoidance_state == 3: # 步驟3: 左轉 (切回)
            SpeedControl(SPEED_AVOID_STRAIGHT, -SPEED_AVOID_TURN)
            if time_elapsed > AVOID_TURN_TIME:
                avoidance_state = 4
                avoidance_timer = current_time
                
        elif avoidance_state == 4: # 步驟4: 直行 (切回後的緩衝)
            SpeedControl(SPEED_AVOID_STRAIGHT, 0)
            if time_elapsed > AVOID_STRAIGHT_TIME:
                avoidance_state = 5
                avoidance_timer = current_time
                
        elif avoidance_state == 5: # 步驟5: 左轉 (回正車身)
            SpeedControl(SPEED_AVOID_STRAIGHT, -SPEED_AVOID_TURN)
            if time_elapsed > AVOID_TURN_TIME:
                avoidance_state = 6
                avoidance_timer = current_time
                
        elif avoidance_state == 6: # 步驟6: 結束避障，稍微暫停或直行
             # 這裡可以直接設回 0，或者給一點緩衝時間
            avoidance_state = 0 
            robot.stop()
            
        # 更新畫面後直接返回，不執行下方的巡線邏輯
        _, frame_encoded = cv2.imencode('.jpg', cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB))
        image_widget.value = frame_encoded.tobytes()
        return 

    # ==========================================
    # --- C. 正常模式：偵測與巡線 ---
    # ==========================================
    
    # 1. 偵測障礙物 (紅色)
    mask_obstacle = cv2.inRange(frame, OBSTACLE_LOWER_RGB, OBSTACLE_UPPER_RGB)
    contours_obs, _ = cv2.findContours(mask_obstacle, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours_obs:
        largest_obs = max(contours_obs, key=cv2.contourArea)
        if cv2.contourArea(largest_obs) > min_area_slider.value:
            M_obs = cv2.moments(largest_obs)
            if M_obs['m00'] > 0:
                cx_obs = int(M_obs['m10'] / M_obs['m00'])
                cy_obs = int(M_obs['m01'] / M_obs['m00'])
                
                # 畫出障礙物 (紅色圓圈)
                cv2.circle(bgr_frame, (cx_obs, cy_obs), 15, (0, 0, 255), 3)
                
                # 判斷是否觸發避障 (距離夠近 + 啟用尋跡模式)
                if cy_obs > OBS_TRIGGER_Y and tracking_toggle.value:
                    print("偵測到障礙物！啟動避障程序...")
                    avoidance_state = 1      # 進入狀態機
                    avoidance_timer = time.time()
                    # 這裡不需要 return，讓畫面更新一次再進入下一幀的狀態機處理

    # 2. 偵測巡線目標 (黃色)
    lower_target = np.array([r_lower.value, g_lower.value, b_lower.value])
    upper_target = np.array([r_upper.value, g_upper.value, b_upper.value])
    mask_target = cv2.inRange(frame, lower_target, upper_target)
    contours_target, _ = cv2.findContours(mask_target, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    closest_centroid = None
    
    if contours_target:
        largest_cnt = max(contours_target, key=cv2.contourArea)
        if cv2.contourArea(largest_cnt) > min_area_slider.value:
            M = cv2.moments(largest_cnt)
            if M['m00'] > 0:
                cx = int(M['m10'] / M['m00'])
                cy = int(M['m01'] / M['m00'])
                closest_centroid = (cx, cy)
                
                # 畫出巡線目標 (綠色實心圓)
                cv2.circle(bgr_frame, (cx, cy), 10, (0, 255, 0), -1)
                
                # 顯示誤差線 (視覺輔助)
                cv2.line(bgr_frame, (frame_center_x, cy), (cx, cy), (255, 255, 0), 2)

    # ==========================================
    # --- D. 馬達控制 (PID 的 P 控制) ---
    # ==========================================
    if tracking_toggle.value:
        if closest_centroid:
            # 計算誤差 (Error)
            error = closest_centroid[0] - frame_center_x
            
            # 轉向比例 (P-gain)，這裡的 100 是放大係數，可調整
            d_in = (error / width) * 100 
            
            # 呼叫你的速度控制函數
            SpeedControl(current_speed, d_in)
        else:
            # 找不到線時，選擇 停止 或 原地旋轉搜尋 (這裡設為停止)
            robot.stop()
    else:
        # 手動模式或未啟用
        robot.stop()

    # ==========================================
    # --- E. 更新畫面 ---
    # ==========================================
    # 畫出 ROI 框 (視覺輔助)
    cv2.rectangle(bgr_frame, (roi_x, roi_y), (roi_x + roi_w, roi_y + roi_h), (0, 0, 255), 1)
    
    # 更新 Image Widget
    _, frame_encoded = cv2.imencode('.jpg', cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB))
    image_widget.value = frame_encoded.tobytes()
    
    # 更新 Binary Widget (除錯用)
    mask_rgb = cv2.cvtColor(mask_target, cv2.COLOR_GRAY2RGB)
    _, binary_encoded = cv2.imencode('.jpg', mask_rgb)
    binary_widget.value = binary_encoded.tobytes()

print('Cell 3: 優化版影像處理函數已定義 (含狀態機)')